In [ ]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.4.0'
spark_version = 'spark-3.4.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q https://archive.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done


In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [ ]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
sales_df = spark.read.csv(
    SparkFiles.get("home_sales_revised.csv"),
    sep = ',',
    header = True
)

sales_df
sales_df.createOrReplaceTempView("sales_view")

In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

four_avg = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM sales_view WHERE bedrooms = 4 GROUP BY date_built")
four_avg.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    296050.24|
|      2012|    298233.42|
|      2017|    296576.69|
|      2014|    299073.89|
|      2013|    299999.39|
|      2011|     302141.9|
|      2015|    307908.86|
|      2010|    296800.75|
+----------+-------------+



In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

three_avg = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM sales_view WHERE bedrooms = 3 AND bathrooms = 3 GROUP BY date_built")
three_avg.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|    290555.07|
|      2012|    293683.19|
|      2017|    292676.79|
|      2014|    290852.27|
|      2013|    295962.27|
|      2011|    291117.47|
|      2015|     288770.3|
|      2010|    292859.62|
+----------+-------------+



In [ ]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

querry_3 = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM sales_view WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built")
querry_3.show()

+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2016|     293965.1|
|      2012|    307539.97|
|      2017|    280317.58|
|      2014|    298264.72|
|      2013|    303676.79|
|      2011|    276553.81|
|      2015|    297609.97|
|      2010|    285010.22|
+----------+-------------+



In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

rating_view = spark.sql("SELECT view, ROUND(AVG(price), 2) FROM sales_view GROUP BY view HAVING ROUND(AVG(price),2) >= 350000")
rating_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.44202542304992676 seconds ---


In [ ]:
# 7. Cache the the temporary table home_sales.
spark.catalog.cacheTable("sales_view")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('sales_view')

True

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

rating_view2 = spark.sql("SELECT view, ROUND(AVG(price), 2) FROM sales_view GROUP BY view HAVING ROUND(AVG(price),2) >= 350000")
rating_view2.show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  87|           1072285.2|
|  73|           752861.18|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 2.0617103576660156 seconds ---


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
sales_df.write.partitionBy("date_built").parquet('sales_view', mode='overwrite')

In [ ]:
# 11. Read the parquet formatted data.
parquet_df = spark.read.parquet('sales_view')

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_df.createOrReplaceTempView("sales_view")

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()

par_view = spark.sql("SELECT view, ROUND(AVG(price), 2) FROM sales_view GROUP BY view HAVING ROUND(AVG(price),2) >= 350000")
par_view.show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+--------------------+
|view|round(avg(price), 2)|
+----+--------------------+
|  51|           788128.21|
|  54|           798684.82|
|  69|           750537.94|
|  73|           752861.18|
|  87|           1072285.2|
|  64|           767036.67|
|  59|            791453.0|
|  85|          1056336.74|
|  52|           733780.26|
|  71|            775651.1|
|  98|          1053739.33|
|  99|          1061201.42|
|  96|          1017815.92|
| 100|           1026669.5|
|  70|           695865.58|
|  61|           746877.59|
|  75|          1114042.94|
|  78|          1080649.37|
|  89|          1107839.15|
|  77|          1076205.56|
+----+--------------------+
only showing top 20 rows

--- 0.8778567314147949 seconds ---


In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table sales_view")

DataFrame[]

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('sales_view')

False